In [6]:
import sys
import os
import json
import pandas as pd



def get_malicious(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list_ma = []
    df_list_ph = []
    for i in file_list:
        local_path = root_path + i
        
        filepath = os.path.join(local_path ,'dumped_result.ma.csv')
        df_list_ma.append( pd.read_csv(filepath))
        
        filepath = os.path.join(local_path ,'dumped_result.ph.csv')
        df_list_ph.append( pd.read_csv(filepath))
        
        print local_path
        
    df_ma = pd.concat(df_list_ma, axis = 0)
    df_ph = pd.concat(df_list_ph, axis = 0)
    return df_ma, df_ph

def get_total_sample(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    total_url = set()
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.txt')
        f = open(filepath)
        recs = [json.loads(rec) for rec in f]
        for rec in recs:
            for k, v in rec.iteritems():
                rec[k] = json.loads(v)
        for rec in recs:
            total_url.add(rec['url'])
        print filepath, len(total_url)
        f.close()
    print "Total URL Count:\t%d" %len(total_url)
    return total_url

def get_malware_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.ma.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    df = df.loc[df["platform"] == 5003]
    df = df.loc[(df["report_type"] == 1).values + (df["report_type"] == 2).values]
    return df

def covert_and_dump(df):
    urls = list(set(df.url))
    print len(urls)
    return pd.DataFrame({"url": urls})

In [7]:
file_list = ["0301","0302","0306","0307","0311","0312","0313","0314","0315","0318","0319",
             "0320","0321","0322","0323","0325","0326","0327","0328","0330","0401","0402", 
             "0410","0411","0412","0415","0416","0417","0418","0419"]
total_url = get_total_sample(file_list)
df_ma, df_ph = get_malicious(file_list)

/home/sparrow/sparrow/data/sangfor/180301/dumped_result.txt 100000
/home/sparrow/sparrow/data/sangfor/180302/dumped_result.txt 162920
/home/sparrow/sparrow/data/sangfor/180306/dumped_result.txt 219430
/home/sparrow/sparrow/data/sangfor/180307/dumped_result.txt 267294
/home/sparrow/sparrow/data/sangfor/180311/dumped_result.txt 334200
/home/sparrow/sparrow/data/sangfor/180312/dumped_result.txt 400270
/home/sparrow/sparrow/data/sangfor/180313/dumped_result.txt 444164
/home/sparrow/sparrow/data/sangfor/180314/dumped_result.txt 486715
/home/sparrow/sparrow/data/sangfor/180315/dumped_result.txt 528351
/home/sparrow/sparrow/data/sangfor/180318/dumped_result.txt 590172
/home/sparrow/sparrow/data/sangfor/180319/dumped_result.txt 651913
/home/sparrow/sparrow/data/sangfor/180320/dumped_result.txt 691585
/home/sparrow/sparrow/data/sangfor/180321/dumped_result.txt 728972
/home/sparrow/sparrow/data/sangfor/180322/dumped_result.txt 767933
/home/sparrow/sparrow/data/sangfor/180323/dumped_result.txt 80

In [8]:
safe_url = list(set(total_url) - set(df_ma.url) - set(df_ph.url))
safe_url = [_.encode('utf-8') for _ in safe_url]
df_safe = pd.DataFrame({'url': safe_url})
print "Safe URL Count:\t%d" %len(df_safe)
df_safe.to_csv("/home/sparrow/sparrow/data/sangfor/safe_0419.csv")

Safe URL Count:	1461039


In [5]:
df = covert_and_dump(df_ma)
df_ma.to_csv("/home/sparrow/sparrow/data/sangfor/malware_all_0419.csv")

39798


In [19]:
file_list = ["0410", "0411", "0412", "0415", "0416"]
df = get_malware_data(file_list)
df.to_csv("/home/sparrow/sparrow/data/sangfor/malware0416.csv")

/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416


# Get EDR data

In [12]:
def get_malware_edr_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.edr.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    return df

In [13]:
file_list = ["0401", "0402", "0410", "0411", "0412", "0415", "0416", "0417", "0418", "0419"]

In [15]:
df = get_malware_edr_data(file_list)

/home/sparrow/sparrow/data/sangfor/180401
/home/sparrow/sparrow/data/sangfor/180402
/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416
/home/sparrow/sparrow/data/sangfor/180417
/home/sparrow/sparrow/data/sangfor/180418
/home/sparrow/sparrow/data/sangfor/180419
